In [1]:
import os

path_ref = "/data/public/PATH-DT-MSU/SR_patches_10x_20x_40x/WSS1_test/40x/"
files = os.listdir(path_ref)
images_ref = [file for file in files if file.endswith(('jpeg', 'png', 'jpg'))]
images_ref.sort()
# print(images_ref)

In [2]:
path_res_before = "res_before_ft_WSS1_test/"
files = os.listdir(path_res_before)
images_res_before = [file for file in files if file.endswith(('jpeg', 'png', 'jpg'))]
images_res_before.sort()
# print(images_res_before)

In [3]:
path_res_after = "res_after_ft_WSS1_test/"
files = os.listdir(path_res_after)
images_res_after = [file for file in files if file.endswith(('jpeg', 'png', 'jpg'))]
images_res_after.sort()
# print(images_res_after)

In [4]:
# Caculate mean of PSNR and SSIM of BEFORE

In [5]:
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
from tqdm import tqdm

def Mean_PSNR_SSIM(path_ref, images_ref, path_res, images_res):
    sum_psnr = 0
    sum_ssim = 0 
    max_iter = 1000
    iter = 0 
    if len(images_ref) != len(images_res):
        print("Error: The two lists have different lengths.")
    else:
        for ref, sr in tqdm(zip(images_ref, images_res), total=len(images_ref), desc="Processing images"):

            im_ref = cv2.imread(path_ref + ref)
            im_model = cv2.imread(path_res + sr)
            im_ref = im_ref[:im_model.shape[0], :im_model.shape[1], :]

            # 计算PSNR
            mse = np.mean((im_ref - im_model) ** 2)
            if mse == 0:
                psnr = float('inf')
            else:
                psnr = 10 * np.log10(255.0 ** 2 / mse)

            im_ref = cv2.cvtColor(im_ref, cv2.COLOR_BGR2RGB)
            im_model = cv2.cvtColor(im_model, cv2.COLOR_BGR2RGB)

            # Compute the Structural Similarity Index (SSIM) between the two images for each color channel
            ssim_r = ssim(im_ref[:,:,0], im_model[:,:,0])
            ssim_g = ssim(im_ref[:,:,1], im_model[:,:,1])
            ssim_b = ssim(im_ref[:,:,2], im_model[:,:,2])

            # Average the SSIM values for all channels
            ssim_avg = np.mean([ssim_r, ssim_g, ssim_b])

            sum_psnr += psnr
            sum_ssim += ssim_avg
            iter = iter + 1
            if iter == max_iter:
                break 

    return sum_psnr / len(images_ref), sum_ssim / len(images_ref)


In [6]:
Mean_PSNR_SSIM(path_ref,images_ref,path_res_before,images_res_before)

Processing images:  36%|███▋      | 999/2747 [33:23<58:26,  2.01s/it]


(10.343339123061343, 0.30731204784478505)

In [7]:
Mean_PSNR_SSIM(path_ref,images_ref,path_res_after,images_res_after)

Processing images:  36%|███▋      | 999/2747 [33:22<58:24,  2.00s/it]


(13.26659374411932, 0.3080885933498759)